# AutoEncoder

+ AutoEncoder is a type of neural network where the output layer has the same dimensionality as the input layer.
+ An autoencoder replicates the data from the input to the output in an unsupervised manner and is therefore sometimes referred to as a replicator neural network.
+ **Applications**
    - Data Compression
    - Image Denoising
    - Dimensionality Reduction
    - Feature Extraction
    - Image Generation
    - Image colourisation

## Importing Libraries

In [1]:
import numpy as np
import tensorflow as tf
from tqdm.notebook import tqdm

## Loading the Dataset

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [3]:
print('X-train:', x_train.shape)
print('Y-train:', y_train.shape)
print('X-test: ', x_test.shape)
print('Y-test: ', y_test.shape)

X-train: (60000, 28, 28)
Y-train: (60000,)
X-test:  (10000, 28, 28)
Y-test:  (10000,)


+ We will not be needing Y_labels for our model.

## Normalizing the Images

In [4]:
x_train = x_train/255.
x_test = x_test/255.

## Customizing the Dimensons

+ Tensor format: (batch_size, height, width, depth)

In [5]:
x_train = np.expand_dims(x_train, 3)
x_test = np.expand_dims(x_test, 3)

## Shuffling & creating batches of data

In [7]:
train_dataset = (tf.data.Dataset.from_tensor_slices(x_train).shuffle(train_size).batch(batch_size))
test_dataset = (tf.data.Dataset.from_tensor_slices(x_test).shuffle(test_size).batch(batch_size))

## Types of AutoEncoders

### Vanilla AutoEncoder

+ In its simplest form, the autoencoder is a three layers net, i.e. a neural net with one hidden layer.
+ On the basis of number of units in hidden layer, it can be further classified as-
    - **UnderComplete AutoEncoder**: less units in hidden layer than input/output layer
    - **OverComplete AutoEncoder**: more units in hidden layer than input/output layer
+ The input and output are the same, and we learn how to reconstruct the input, for example using the adam optimizer and the mean squared error loss function.
+ This can be extended by multiple hidden layers (i.e, **Multilayer AutoEncoder**)
![UnderComplete AutoEncoder](https://miro.medium.com/max/1088/1*eE-jG_gXajuGcZYHFasAmA.png)

#### Defining Model

In [333]:
class VanillaAutoEncoder(tf.keras.Model):
    
    def __init__(self, intermediate_dim, original_dim):
        super(VanillaAutoEncoder, self).__init__(name='VanillaAutoEncoder')
        
        self.encoder = tf.keras.Sequential([
            tf.keras.Input(shape=original_dim, name='EncoderInput', dtype='float32'),
            tf.keras.layers.Dense(units=intermediate_dim, activation='relu', name='EncoderOutput', dtype='float32')
        ], name='Encoder')
        
        self.decoder = tf.keras.Sequential([
            tf.keras.Input(shape=intermediate_dim, name='DecoderInput', dtype='float32'),
            tf.keras.layers.Dense(units=original_dim, activation='softmax', name='DecoderOutput', dtype='float32')
        ], name='Decoder')
        
        print(super(VanillaAutoEncoder, self).build((None, original_dim)))
  
    def call(self, input_features):
        input_features = tf.cast(input_features, dtype='float32')
        coded_features = self.encoder(input_features)
        reconstructed = self.decoder(coded_features)
        return reconstructed
        
    def encode(self, input_features):
        return self.encoder(input_features)
    
    def decode(self, coded_features):
        return self.decoder(coded_features)

#### Defining Loss Function

In [334]:
def loss(model, original):
    reconstruction_error = tf.reduce_mean(tf.square(tf.subtract(model(original), tf.cast(original, dtype='float32'))))
    return reconstruction_error

#### Defining Training Process

In [335]:
@tf.function
def train(loss, model, optimizer, original):
    with tf.GradientTape() as tape:
        gradients = tape.gradient(loss(model, original), model.trainable_variables)
        gradient_variables = zip(gradients, model.trainable_variables)
        optimizer.apply_gradients(gradient_variables)

#### Defining some HyperParameters

In [363]:
train_size = x_train.shape[0]
test_size = x_test.shape[0]
batch_size = 32
train_batch_size = train_size//batch_size
test_batch_size = test_size//batch_size
epochs = 5
learning_rate = 0.01
intermediate_dim = 64

#### Preparing Data for Model

In [336]:
flat_x_train = x_train.reshape(x_train.shape[0], x_train.shape[1]*x_train.shape[2]*x_train.shape[3]).astype(np.float32)
flat_x_test = x_test.reshape(x_test.shape[0], x_test.shape[1]*x_test.shape[2]*x_test.shape[3]).astype(np.float32)

In [337]:
train_dataset = (tf.data.Dataset.from_tensor_slices(flat_x_train).shuffle(train_size).batch(batch_size))
test_dataset = (tf.data.Dataset.from_tensor_slices(flat_x_test).shuffle(test_size).batch(batch_size))

#### Defining Optimizer

In [339]:
opt = tf.optimizers.Adam(learning_rate=learning_rate)

#### Initializing Model

In [340]:
autoencoder = VanillaAutoEncoder(intermediate_dim=intermediate_dim, original_dim=flat_x_train.shape[1])
autoencoder.summary()

None
Model: "VanillaAutoEncoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Encoder (Sequential)         (None, 64)                50240     
_________________________________________________________________
Decoder (Sequential)         (None, 784)               50960     
Total params: 101,200
Trainable params: 101,200
Non-trainable params: 0
_________________________________________________________________


In [341]:
autoencoder.encoder.summary()

Model: "Encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
EncoderOutput (Dense)        (None, 64)                50240     
Total params: 50,240
Trainable params: 50,240
Non-trainable params: 0
_________________________________________________________________


In [342]:
autoencoder.decoder.summary()

Model: "Decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
DecoderOutput (Dense)        (None, 784)               50960     
Total params: 50,960
Trainable params: 50,960
Non-trainable params: 0
_________________________________________________________________


#### Training Model

In [365]:
for epoch in range(epochs):
    for step, batch_input in tqdm(enumerate(train_dataset), desc='Epoch - '+str(epoch+1), total=train_batch_size):
        train(loss, autoencoder, opt, batch_input)
    print('Loss:', float(loss(autoencoder, batch_input)))


Loss: 0.1221349686384201



Loss: 0.11579706519842148



Loss: 0.10793193429708481



Loss: 0.11771609634160995



Loss: 0.11351384967565536


### Convolutional AutoEncoder (CAE)

+ CAE learn to encode the input image into a set of simple features and then reconstruct the input from them.
+ In this type of autoencoder, encoder layers are known as convolution layers and decoder layers are also called deconvolution layers. The deconvolution side is also known as upsampling or transpose convolution.
![Convolutional AutoEncoder](https://iq.opengenus.org/content/images/2019/07/a5.png)

### Sparse AutoEncoder

+ Sparse autoencoders have hidden nodes greater than input nodes. They can still discover important features from the data.
+ Sparsity constraint is introduced on the hidden layer. This is to prevent output layer copy input data.
+ Sparse autoencoders have a sparsity penalty, Ω(h), a value close to zero but not zero. Sparsity penalty is applied on the hidden layer in addition to the reconstruction error. This prevents overfitting.
+ We could, for example, add a reguralization term in the loss function. Doing this will make our autoencoder learn sparse representation of data.
+ Another way is to take the highest activation values in the hidden layer and zero out the rest of the hidden nodes. This prevents autoencoders to use all of the hidden nodes at a time and forcing only a reduced number of hidden nodes to be used.
+ As we activate and inactivate hidden nodes for each row in the dataset. Each hidden node extracts a feature from the data
![Sparse AutoEncoder](https://miro.medium.com/max/990/1*19oadmQay1n7VNarX5sIPA.png)

### Denoising AutoEncoder

+ Denoising autoencoders add some noise to the input image and learn to remove it. Thus avoiding to copy the input to the output without learning features about the data. These autoencoders take a partially corrupted input while training to recover the original undistorted input.
+ The model learns a vector field for mapping the input data towards a lower-dimensional manifold which describes the natural data to cancel out the added noise. By this means, the encoder will extract the most important features and learn a more robust representation of the data.
![Denoising AutoEncoders](https://iq.opengenus.org/content/images/2019/07/a1.png)

### Contractive AutoEncoder

+ The objective of a contractive autoencoder is to have a robust learned representation which is less sensitive to small variation in the data. Robustness of the representation for the data is done by applying a penalty term to the loss function.
+ Contractive autoencoder is another regularization technique just like sparse and denoising autoencoders. However, this regularizer corresponds to the Frobenius norm of the Jacobian matrix of the encoder activations with respect to the input. Frobenius norm of the Jacobian matrix for the hidden layer is calculated with respect to input and it is basically the sum of square of all elements.
+ Contractive autoencoder is a better choice than denoising autoencoder to learn useful feature extraction.
![Contractive AutoEncoder](https://iq.opengenus.org/content/images/2019/07/a4.png)
![Loss function with penalty term — Frobenius norm of the Jacobian matrix](https://miro.medium.com/max/326/1*FjATD2EW-bFB4vTQ6ikR4w.png)

### Deep AutoEncoder

+ Deep Autoencoders consist of two identical deep belief networks, oOne network for encoding and another for decoding. Typically deep autoencoders have 4 to 5 layers for encoding and the next 4 to 5 layers for decoding. We use unsupervised layer by layer pre-training for this model.
+ The layers are Restricted Boltzmann Machines which are the building blocks of deep-belief networks.
+ Deep autoencoders are useful in topic modeling, or statistically modeling abstract topics that are distributed across a collection of documents. They are also capable of compressing images into 30 number vectors.
![Deep AutoEncoder](https://miro.medium.com/max/1400/1*QM1b0gbKdMowkmyvO95DFA.png)

### Convolutional Variational AutoEncoder (VAE)

+ VAE is a probabilistic take on the autoencoder, a model which takes high dimensional input data compress it into a smaller representation.
+ This type of autoencoder can generate new images just like GANs.
+ Unlike a traditional autoencoder, which maps the input onto a latent vector, a VAE maps the input data into the parameters of a probability distribution, such as the mean and variance of a Gaussian.
+ They use a variational approach for latent representation learning, which results in an additional loss component and a specific estimator for the training algorithm called the **Stochastic Gradient Variational Bayes** estimator.
+ The probability distribution of the latent vector of a variational autoencoder typically matches the training data much closer than a standard autoencoder. 
+ This approach produces a continuous, structured latent space, which is useful for image generation.
+ As VAEs are much more flexible and customisable in their generation behaviour than GANs, they are suitable for art generation of any kind.
![Convolutional Variational AutoEncoder](https://iq.opengenus.org/content/images/2019/07/a6.png)

In [1]:
class CVAE(tf.keras.Model):
    """Convolutional Variational AutoEncoder."""
    
    def __init__(self, latent_dim):
        super(CVAE, self).__init__()
        self.latent_dim = latent_dim
        
        self.encoder = tf.keras.Sequential([
                tf.keras.layers.InputLayer(input_shape=(28, 28, 1)),
                tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=(2, 2), activation='relu'),
                tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=(2, 2), activation='relu'),
                tf.keras.layers.Flatten(),
                # No activation
                tf.keras.layers.Dense(latent_dim + latent_dim),
        ])

        self.decoder = tf.keras.Sequential([
                tf.keras.layers.InputLayer(input_shape=(latent_dim,)),
                tf.keras.layers.Dense(units=7*7*32, activation=tf.nn.relu),
                tf.keras.layers.Reshape(target_shape=(7, 7, 32)),
                tf.keras.layers.Conv2DTranspose(filters=64, kernel_size=3, strides=2, padding='same', activation='relu'),
                tf.keras.layers.Conv2DTranspose(filters=32, kernel_size=3, strides=2, padding='same', activation='relu'),
                # No activation
                tf.keras.layers.Conv2DTranspose(filters=1, kernel_size=3, strides=1, padding='same'),
        ])

    
    @tf.function
    def sample(self, eps=None):
        if eps is None:
            eps = tf.random.normal(shape=(100, self.latent_dim))
        return self.decode(eps, apply_sigmoid=True)

    
    def encode(self, x):
        mean, logvar = tf.split(self.encoder(x), num_or_size_splits=2, axis=1)
        return mean, logvar

    
    def reparameterize(self, mean, logvar):
        eps = tf.random.normal(shape=mean.shape)
        return eps * tf.exp(logvar * .5) + mean

    
    def decode(self, z, apply_sigmoid=False):
        logits = self.decoder(z)
        if apply_sigmoid:
            probs = tf.sigmoid(logits)
            return probs
        return logits

NameError: name 'tf' is not defined

In [18]:
optimizer = tf.keras.optimizers.Adam(1e-4)


def log_normal_pdf(sample, mean, logvar, raxis=1):
    log2pi = tf.math.log(2. * np.pi)
    return tf.reduce_sum(-.5 * ((sample - mean) ** 2. * tf.exp(-logvar) + logvar + log2pi), axis=raxis)


def compute_loss(model, x):
    mean, logvar = model.encode(x)
    z = model.reparameterize(mean, logvar)
    x_logit = model.decode(z)
    cross_ent = tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=x)
    logpx_z = -tf.reduce_sum(cross_ent, axis=[1, 2, 3])
    logpz = log_normal_pdf(z, 0., 0.)
    logqz_x = log_normal_pdf(z, mean, logvar)
    return -tf.reduce_mean(logpx_z + logpz - logqz_x)


@tf.function
def train_step(model, x, optimizer):
    """Executes one training step and returns the loss.

    This function computes the loss and gradients, and uses the latter to
    update the model's parameters.
    """
    with tf.GradientTape() as tape:
        loss = compute_loss(model, x)
    
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

In [19]:
epochs = 10
# set the dimensionality of the latent space to a plane for visualization later
latent_dim = 2
num_examples_to_generate = 16

# keeping the random vector constant for generation (prediction) so
# it will be easier to see the improvement.
random_vector_for_generation = tf.random.normal(shape=[num_examples_to_generate, latent_dim])
model = CVAE(latent_dim)

+ Note: **Incomplete**
    - https://www.tensorflow.org/tutorials/generative/cvae